In [34]:
# load library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import zipfile
import urllib.request
import io
import warnings
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
warnings.filterwarnings('ignore')

In [31]:
# load raw and processed data
train_merged_df_latest = pd.read_csv('https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/data_processing/train_merged_latest_feat.csv')
train_merged_df_median = pd.read_csv('https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/data_processing/train_merged_median_feat.csv')
test_merged_df_latest = pd.read_csv('https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/data_processing/test_merged_latest_feat.csv')
test_merged_df_median = pd.read_csv('https://github.com/zxyao5148/STAT3612_2023_1A_GroupProject/raw/main/data_processing/test_merged_median_feat.csv')

In [33]:
train_merged_df_latest.head()

,id,subject_id,hadm_id,admittime,dischtime,deathtime,StudyDate,StudyTime,readmitted_within_30days,LoS,...,CARDIAC DRUGS,PRE-NATAL VITAMINS,ANESTHETICS,ANTIBIOTICS,ANTIHYPERGLYCEMICS,SEDATIVE/HYPNOTICS,ANTIDOTES,AUTONOMIC DRUGS,VITAMINS,BIOLOGICALS
0,17195991_23542772,17195991,23542772,2110-01-11 22:47:00,2110-01-18 10:25:00,NaN,2110-01-16,90654.546,0,-0.405473,...,-0.379713,-0.013935,-0.455273,-0.449292,-0.318458,-0.418357,-0.047707,2.185809,-0.381595,-0.304994
1,13721591_20342223,13721591,20342223,2110-02-09 18:13:00,2110-02-22 20:51:00,NaN,2110-02-19,41948.468,0,-0.026227,...,-0.379713,-0.013935,0.210369,-0.130508,-0.232069,-0.418357,-0.047707,-0.306970,-0.381595,-0.304994
2,19170541_22178312,19170541,22178312,2110-02-28 21:48:00,2110-03-12 17:47:00,NaN,2110-03-11,81842.812,0,-0.134583,...,-0.379713,-0.013935,0.210369,-0.289900,-0.318458,-0.418357,-0.047707,-0.306970,-0.381595,-0.304994
3,15554295_27705504,15554295,27705504,2110-03-09 03:54:00,2110-05-18 11:34:00,NaN,2110-05-04,60653.312,0,3.061922,...,-0.379713,-0.013935,0.210369,1.463409,-0.232069,3.126321,-0.047707,1.354882,-0.381595,-0.304994
4,17643026_29919541,17643026,29919541,2110-03-25 11:15:00,2110-03-29 17:17:00,NaN,2110-03-28,140521.453,0,-0.513830,...,-0.379713,-0.013935,-0.455273,-0.608684,-0.059292,-0.418357,-0.047707,-0.306970,-0.381595,1.561410


In [32]:
# separate response and features
X_train = train_merged_df_latest.iloc[:, 3:]
y_train = train_merged_df_latest['readmitted_within_30days']
X_test = test_merged_df_latest.iloc[:, 3:]
y_test = test_merged_df_latest['readmitted_within_30days']

KeyError: 'readmitted_within_30days'

In [4]:
# logistic regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_accuracy_train = accuracy_score(y_train, lr.predict(X_train))
lr_accuracy_test = cross_val_score(lr, X_train, y_train, cv=5)
print(f'LR Accuracy (train): {lr_accuracy_train:.4f}')
print(f'LR Accuracy (cv): {lr_accuracy_test.mean():.4f}')

Logistic Regression Training Error: 0.8522
Logistic Regression Testing Error: 0.8439


In [5]:
# Linear discriminant analysis
lda = LinearDiscriminantAnalysis()
lda_accuracy_train = accuracy_score(y_train, lda.predict(X_train))
lda_accuracy_test = cross_val_score(lda, X_train, y_train, cv=5)
print(f'LDA Accuracy (train): {lda_accuracy_train:.4f}')
print(f'LDA Accuracy (cv): {lda_accuracy_test.mean():.4f}')

Linear Discriminant Analysis Training Accuracy: 0.8530
Linear Discriminant Analysis Testing Accuracy: 0.8409


In [6]:
# Quadratic discriminant analysis
from sklearn.model_selection import cross_val_score
qda = QuadraticDiscriminantAnalysis()
qda_accuracy_train = accuracy_score(y_train, qda.predict(X_train))
qda_accuracy_test = cross_val_score(qda, X_train, y_train, cv=5)
print(f'QDA Accuracy (train): {qda_accuracy_train:.4f}')
print(f'QDA Accuracy (cv): {qda_accuracy_test.mean():.4f}')

Quadratic Discriminant Analysis Training Accuracy: 0.1940
Quadratic Discriminant Analysis Testing Accuracy: 0.8215


In [7]:
# Gaussian naive bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_accuracy_train = accuracy_score(y_train, gnb.predict(X_train))
gnb_accuracy_test = cross_val_score(gnb, X_train, y_train, cv=5)
print(f'GNB Accuracy (train): {gnb_accuracy_train:.4f}')
print(f'GNB Accuracy (cv): {gnb_accuracy_test.mean():.4f}')

Gaussian Naive Bayes Training Accuracy: 0.1984
Gaussian Naive Bayes Testing Accuracy: 0.8215


In [29]:
# K-nearest neighbors
param_grid = {'n_neighbors': [3, 5, 7, 9, 11]}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(f'KNN Best Parameters: {grid_search.best_params_}')
print(f'KNN Accuracy (train): {accuracy_score(y_train, grid_search.predict(X_train)):.4f}')
print(f'KNN Accuracy (cv): {grid_search.best_score_:.4f}')

AttributeError: 'NoneType' object has no attribute 'split'

In [43]:
# Support vector machine
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'degree': [2, 3, 4],
    'gamma': ['scale', 'auto']
}
svm = SVC()
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(f'SVM Best Parameters: {grid_search.best_params_}')
print(f'SVM Accuracy (train): {accuracy_score(y_train, grid_search.predict(X_train)):.4f}')
print(f'SVM Accuracy (cv): {grid_search.best_score_:.4f}')

Support Vector Machine Training Accuracy: 0.8570
Support Vector Machine Testing Accuracy: 0.8430


In [44]:
# Decision Tree
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4]
}
dt = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(f'DT Best Parameters: {grid_search.best_params_}')
print(f'DT Accuracy (train): {accuracy_score(y_train, grid_search.predict(X_train)):.4f}')
print(f'DT Accuracy (cv): {grid_search.best_score_:.4f}')

Decision Tree Training Accuracy: 1.0000
Decision Tree Testing Accuracy: 0.7591


In [45]:
# Bagging
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_samples': [0.5, 0.7, 0.9],
    'max_features': [0.5, 0.7, 0.9]
}
grid_search = GridSearchCV(estimator=BaggingClassifier(), param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(f'Bagging Best Parameters: {grid_search.best_params}')
print(f'Bagging Accuracy (train): {accuracy_score(y_train, grid_search.predict(X_train)):.4f}') 
print(f'Bagging Accuracy (cv): {grid_search.best_score_:.4f}')

Bagging Training Accuracy: 0.9828
Bagging Testing Accuracy: 0.8430


In [46]:
# Random forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4]
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(f'RF Best Parameters: {grid_search.best_params_}')
print(f'RF Accuracy (train): {accuracy_score(y_train, grid_search.predict(X_train)):.4f}') 
print(f'RF Accuracy (cv): {grid_search.best_score_:.4f}')

Random Forests Training Accuracy: 0.9999
Random Forests Testing Accuracy: 0.8589


In [47]:
# Gradient boosting
param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 3, 4],
    'n_estimators': [50, 100, 200]
}
gb = GradientBoostingClassifier()
grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(f'GB Best Parameters: {grid_search.best_params_}')
print(f'GB Accuracy (train): {accuracy_score(y_train, grid_search.predict(X_train)):.4f}')
print(f'GB Accuracy (cv): {grid_search.best_score_:.4f}')

Gradient Boosting Training Accuracy: 0.8712
Gradient Boosting Testing Accuracy: 0.8624


In [15]:
# AdaBoost
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1]
}
ada = AdaBoostClassifier()
grid_search = GridSearchCV(estimator=ada, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(f'AdaBoost Best Parameters: {grid_search.best_params_}')
print(f'AdaBoost Accuracy (train): {accuracy_score(y_train, grid_search.predict(X_train)):.4f}')
print(f'AdaBoost Accuracy (cv): {grid_search.best_score_:.4f}')

AdaBoost Training Accuracy: 0.8561
AdaBoost Testing Accuracy: 0.8482


In [27]:
# XGBoost
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 500, 1000]
}
xgb = XGBClassifier()
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print(f'XGBoost Best Hyperparameters: {grid_search.best_params_}')
print(f'XGBoost Accuracy (train): {accuracy_score(y_train, grid_search.predict(X_train)):.4f}')
print(f'XGBoost Accuracy (cv): {grid_search.best_score_:.4f}')

XGBoost Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
XGBoost Accuracy (train): 0.9011
XGBoost Accuracy (cv): 0.8641
